## フェーズ1: 盤石な基盤モデルの構築 - 学習プロセスの高度化（Mixup/CutMix） (`feature/mixup-cutmix`)

転移学習による基盤モデルの構築が成功し、高い検証精度を達成しました。次に、モデルの汎化性能をさらに高め、過学習を抑制するために、データ拡張手法であるMixupとCutMixを導入しました。

### やったこと

-   **Mixup/CutMixヘルパー関数の実装 (`src/train.py`)**:
    -   `train.py` 内に `mixup_data` と `cutmix_data` 関数を新しく追加しました。
    -   `mixup_data` は、2つの画像とラベルを線形補間して新しいサンプルを生成します。
    -   `cutmix_data` は、2つの画像の一部を切り貼りして新しい画像を生成し、対応するラベルを混合します。
    -   両関数とも、Beta分布からサンプリングされた $\lambda$ 値を用いて、混合の割合を決定します。
    -   混合されたラベルに対する損失を計算するための `mix_criterion` 関数も追加しました。

-   **`train_model` 関数へのMixup/CutMixの組み込み (`src/train.py`)**:
    -   訓練ループ内で、各バッチ処理の前にランダムにMixupまたはCutMixを適用するロジックを導入しました。
    -   適用されるデータ拡張の種類（MixupまたはCutMix）と、それが適用される確率 (`MIX_PROB`) をハイパーパラメータとして設定できるようにしました。
    -   Mixup/CutMixが適用された場合、生成された混合画像と混合ラベルを用いて損失を計算するように修正しました。

### 動作確認

構築したMixup/CutMixを含む学習スクリプトは、`src/train.py` を直接実行することで動作確認が可能です。

```bash
# srcディレクトリ内から実行
python3 train.py
```
実行すると、K-Fold交差検証が開始され、各フォールドの訓練・検証の進捗が出力されます。Mixup/CutMixの導入により、訓練損失が（通常学習時と比較して）高めに推移する傾向が見られることがありますが、これはモデルがより多様で「難しい」サンプルを学習しているためであり、検証精度が安定し、最終的な汎化性能が向上することが期待されます。

出力例:
(Mixup/CutMixを適用した場合の訓練損失は通常より高くなる傾向があります)
```
認識対象クラス数: 10
全データ数: 2501
Using device: cuda
Using model: convnext_tiny

--- Fold 1/5 ---
  訓練データ数 (Fold 1): 2000
  検証データ数 (Fold 1): 501
Epoch 1/10 (Train): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 63/63 [01:05<00:00,  1.03s/it]
Epoch 1/10 (Validation): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:07<00:00,  2.08it/s]
Epoch 1 | Train Loss: X.XXXX | Val Loss: Y.YYYY | Val Accuracy: Z.ZZ%
... (学習の進捗)
```